In [ ]:
from filesender.benchmark import benchmark
from os import environ
from matplotlib import pyplot as plt

x = list(range(10))
results = benchmark(x, apikey=environ["API_KEY"], base_url=environ["BASE_URL"], recipient=environ["RECIPIENT"], username=environ["USERNAME"])

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, [result.memory for result in results])
ax.yaxis.set_label("Memory Usage (bytes)")
ax.xaxis.set_label("Maximum Concurrency")
fig.suptitle("Memory Usage")

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, [result.time for result in results])
ax.yaxis.set_label("Runtime (seconds)")
ax.xaxis.set_label("Maximum Concurrency")
fig.suptitle("Runtime")